In [1]:
import pandas as pd
  # defining a function for airline demand forecasting
def AirlineForecast(trainingfile,validationfile):
 training=pd.read_csv(trainingfile) 
 validation=pd.read_csv(validationfile)
  #converting the date from string to date format
 training['departure_date'] = pd.to_datetime(training['departure_date']) 
 training['booking_date'] = pd.to_datetime(training['booking_date'])
 validation['booking_date'] = pd.to_datetime(validation['booking_date'])
 validation['departure_date'] = pd.to_datetime(validation['departure_date'])
 valbook=pd.to_datetime(validation['booking_date'])
 valdep=pd.to_datetime(validation['departure_date'])
  # calculating Days of Week
 training['Days']=training['booking_date'].dt.day_name()
 validation['Days']=validation['booking_date'].dt.day_name()
  # calculating days prior 
 daysprior=training['departure_date'] - training['booking_date']      
 valdaysprior=valdep-valbook             
 training['days prior']=daysprior
 validation['days prior']=valdaysprior
  # calculating final demand
 finaldemand=training.apply(lambda row: training[(training['departure_date'] == row['departure_date']) & (training['booking_date'] == training['departure_date']) & (training['booking_date'] >= row['booking_date'])]['cum_bookings'].max(), axis=1)
 finaldemandvalidation=validation.apply(lambda row: validation[(validation['departure_date'] == row['departure_date'])&  (validation['booking_date'] == validation['departure_date'])  & (validation['booking_date'] >= row['booking_date'])]['cum_bookings'].max(), axis=1)
 training['final demand']=finaldemand
 validation['final demand']=finaldemandvalidation
  # calculating forecast for remaining demands
 Forecastsforremainingdemand = training['final demand']-training['cum_bookings'] 
 training['demandforecast']=Forecastsforremainingdemand 
# creating a data frame with columns days prior, demandforecast, and DOW
 df = pd.DataFrame({'days prior': training['days prior'], 'demandforecast': training['demandforecast'], 'DOW': training['Days']})
  # calculating the mean of demand forecasts for particular days prior and DOW to make the mapping 1:1 in the data frame
 d = df.groupby(['days prior', 'DOW']).mean()
  # merging the data frame to the validation data
 validationdata=validation.merge(d, left_on=['days prior', 'Days'], right_on=['days prior', 'DOW'])
  # calculating the final forecast in the validation data by formula forecastforremainigdemand+onthebook
 validationdata['forecast']=validationdata['demandforecast']+validationdata['cum_bookings']
  # calculates the error associated with the final forecast and naive forecast
 Error1= validationdata['final demand']-validationdata['forecast']
 Error2= validationdata['final demand']-validationdata['naive_fcst']  
  # calculating MASE value
 MASE=abs(Error1).sum()/abs(Error2).sum()    
  # Extracts departure date, booking date and forecast from validation data
 finaldata=pd.DataFrame({'departure_date': validationdata['departure_date'],'booking_date':validationdata['booking_date'], 'forecast':validationdata['forecast']})
 return MASE, finaldata
AirlineForecast( "airline_data_training.csv", "airline_data_validation.csv")

(0.643957324451688,
     departure_date booking_date    forecast
 0       2012-11-01   2012-11-01  269.000000
 1       2012-11-08   2012-11-08  476.000000
 2       2012-11-01   2012-10-31  271.090909
 3       2012-11-08   2012-11-07  452.090909
 4       2012-11-01   2012-10-30  271.454545
 ..             ...          ...         ...
 205     2012-11-14   2012-11-02  567.272727
 206     2012-11-07   2012-10-25  563.636364
 207     2012-11-14   2012-11-01  562.636364
 208     2012-11-07   2012-10-24  570.636364
 209     2012-11-14   2012-10-31  551.636364
 
 [210 rows x 3 columns])